In [1]:
import pandas as pd
import functions as fn
import json
import os

In [3]:
### NOTE: only need to run this once - creates a file containing all the processed information ready for annotation
file_path = "./data/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/note/"
df = pd.read_csv(file_path+"discharge.csv")
all_cards = []
for i in range(len(df)):
    processor = fn.AnnotationProcess(df["text"][i])
    cards = processor.extract_HOPC()
    if cards is not None:  # only append cards if it's not None
        all_cards.append(cards)
    # if all_cards has 10000 entries, save and clear the list
    if len(all_cards) == 10000:
        file_num = i // 10000 + 1  # compute the file number
        with open(f'./data/HOPC/HOPC_{file_num}.jsonl', "w") as f:
            for item in all_cards:
                json.dump(item, f)  # write each item on its own line
                f.write('\n')  # add a newline after each item
        all_cards = []  # reset the all_cards list for the next batch

# save remaining cards, if any
if all_cards:
    file_num = len(df) // 10000 + 1  # compute the file number
    with open(f'./data/HOPC/HOPC_{file_num}.jsonl', "w") as f:
        for item in all_cards:
            json.dump(item, f)  # write each item on its own line
            f.write('\n')  # add a newline after each item

In [9]:
dir = "./data"
label_list = [
   "symptoms",
   "negative_symptoms"
]
database = "irAE"
model = "en_core_web_sm"
annotate = fn.ProdigyFunctions(dir, label_list, database, model)

annotate.database_connect()

annotate.prodigy_connect("ner.manual", "./data/HOPC/HOPC_1.json")


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using 2 label(s): symptoms, negative_symptoms


Traceback (most recent call last):
  File "/Users/andrewxu/miniforge3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/andrewxu/miniforge3/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/andrewxu/Documents/2023/MD2/Austin_Research/Code/irAE-detection/env/lib/python3.9/site-packages/prodigy/__main__.py", line 62, in <module>
    controller = recipe(*args, use_plac=True)
  File "cython_src/prodigy/core.pyx", line 374, in prodigy.core.recipe.recipe_decorator.recipe_proxy
  File "/Users/andrewxu/Documents/2023/MD2/Austin_Research/Code/irAE-detection/env/lib/python3.9/site-packages/plac_core.py", line 367, in call
    cmd, result = parser.consume(arglist)
  File "/Users/andrewxu/Documents/2023/MD2/Austin_Research/Code/irAE-detection/env/lib/python3.9/site-packages/plac_core.py", line 232, in consume
    return cmd, self.func(*(args + varargs + extraopts), **kwargs)
  File "/Users

In [2]:
# training
output_dir = "./models/"
ner_data = "irAE"

os.system(f"prodigy train ./models/ --ner irAE --config config.cfg")

ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

========================= Generating Prodigy config =========================
✔ Generated training config

=========================== Initializing pipeline ===========================


[2023-07-03 20:49:55,749] [INFO] Set up nlp object from config
Components: ner
Merging training and evaluation data for 1 components
  - [ner] Training: 68 | Evaluation: 16 (20% split)
Training: 68 | Evaluation: 16
Labels: ner (5)
[2023-07-03 20:49:55,791] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-07-03 20:49:55,793] [INFO] Created vocabulary
[2023-07-03 20:49:55,793] [INFO] Finished initializing nlp object
[2023-07-03 20:49:56,733] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
Components: ner
Merging training and evaluation data for 1 components
  - [ner] Training: 68 | Evaluation: 16 (20% split)
Training: 68 | Evaluation: 16
Labels: ner (5)


✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    218.91    0.00    0.00    0.00    0.00
  3     200       2944.82   7531.03   18.18   28.00   13.46    0.18
  6     400        868.74   1695.97   41.21   43.16   39.42    0.41
  9     600       4138.34   1045.00   47.42   51.11   44.23    0.47
 12     800       4053.03   1051.52   47.25   55.13   41.35    0.47
 16    1000        891.84    673.91   50.78   55.06   47.12    0.51
 19    1200       2365.28    480.07   42.42   44.68   40.38    0.42
 22    1400       1852.96    458.16   41.76   48.72   36.54    0.42
 25    1600       5384.54    421.15   44.57   51.25   39.42    0.45
 29    1800       4456.28    370.64   49.26   50.51   48.08    0.49
 32    2000        577.

0

In [4]:
os.system("prodigy ner.correct irAE ./models/model-last/ ./data/HOPC/HOPC_3.jsonl --label symptoms,negative_symptoms")

Using 2 label(s): symptoms, negative_symptoms
⚠ The model you're using isn't setting sentence boundaries (e.g. via
the parser or sentencizer). This means that incoming examples won't be split
into sentences.

✨  Starting the web server at http://localhost:8080 ...
Open the app in your browser and start annotating!

